Make Fig.2 given cadence and errors set by a given light curve ... 

taking the code from `../code/NOX_...`

In [4]:
import os 
import pandas as pd 
import numpy as np 

import imp
mod = imp.load_source('modules.py', '../code/modules.py')

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table
from astropy.table import  hstack
from astropy.table import  vstack
from astropy.table import unique
from astropy.table import Column
from astropy.table import join

import matplotlib.pyplot as plt
from matplotlib import rcParams 
rcParams['ytick.labelsize'] = 15
rcParams['xtick.labelsize'] = 15
rcParams['axes.labelsize'] = 20
rcParams['axes.linewidth'] = 2
rcParams['font.size'] = 15
rcParams['axes.titlesize'] = 18

outDir = '../dp2/Fig2_simLC_SDSS-PS1/'

Read the first of all SDSS-PS1 light curves ....

In [42]:
%matplotlib inline
# combined light curves
realDir = '../data_products/Combined_LC/'
realLcs =  os.listdir(realDir)

#for i in range(len(realLcs)):
#    if (i % 1000) == 0 : 
#        print(i)

def sim_Fig2_data(lcname):
    # Load the i-th combined real light curve 
    lc_comb = Table.read(realDir+lcname, format='ascii')
    lc_comb.sort('mjd') # sort by mjd .... 
    # keep only the first row if there are two rows that for 
    # some reason have the same MJD ...
    lc_comb = unique(lc_comb, keys='mjd', keep='first')


    # use only SDSS-PS1 points ...
    m = np.in1d(lc_comb['originSurvey'] , 'sdss-ps1'.split('-'))
    lc_comb = lc_comb[m]

    # this gives the time stamps (cadence) and errors
    # to make mag-error connection realistic, 
    # simulate all DRW around 0 mean, and 
    # then add the median light curve magnitude .... 

    t_exp =max(lc_comb['mjd']) - min(lc_comb['mjd']) # days 
    SF_inf = 0.2 # mag 
    sigma = SF_inf / np.sqrt(2)

    rho_min, rho_max,  n_rho = 0.001, 15, 100
    rho_grid = np.logspace(np.log10(rho_min), np.log10(rho_max), n_rho)
    tau_grid = rho_grid * t_exp

    print(' %.2f < tau_in < %.2f  [days]'% (min(tau_grid), max(tau_grid)))
    print('Input  SF_inf:', SF_inf, 'sigma:', sigma)


    # use the cadence of combined SDSS-PS1 light curve 
    # to save space, I am not making long light curves 
    # that would then be subsampled , as I did 
    # for another simulation of SDSS-PS1-ZTF , etc , 
    # 
    t = lc_comb['mjd'].data
    y_err = lc_comb['magerr'].data
    mean_mag = np.mean(lc_comb['mag']) # its all in combined r-band ...
    # calculating the mean of all SDSS-PS1 is fine 

    
    if not os.path.exists(outDir):
        os.makedirs(outDir)

    simulated = {}

    simulated['y_err'] = y_err
    simulated['mjd'] = t
    simulated['originSurvey'] = lc_comb['originSurvey']

    for i,rho_in in enumerate(rho_grid) : 
        tau_in = rho_in * t_exp
        if i % 10 == 0 :  
            print(i)
            #plt.errorbar(t, y_obs+i*0.1, y_err,fmt=".", capsize=0.2,)
        # for each rho, simulate 100 light curves 
        simulated[i] = {}
        simulated[i]['rho_in'] = rho_in
        simulated[i]['tau_in'] = tau_in
        for j in range(100): # 100 
            y_true = mod.sim_DRW_lightcurve(t, SF_inf, tau_in , 0)
            #name = 'DRW_rho-' + str(i).zfill(3)+'_'+str(j).zfill(3)+'.txt'
            # store the observed underlying light curve,  
            # i.e. the DRW signal  +  Gaussian noise + median LC mag 
            # this skips another step as compared to 
            # NOX_Make_lightcurves , which first saves ideal LCs, 
            # and then 'observes' them, but I want to save space 
            # (and time) and quickly see the Fig.2 result 
            # for that particular set of cadence and errors 

            # add the median magnitude of the combined light curve 
            y_true += mean_mag 

            # add the photometric noise corresponding to epochal errors 
            y_obs = y_true +  y_err * np.random.normal(loc=0,scale=1,size=len(y_true))
            # np.savetxt(outDir+name,y, fmt = ' %.7f ')   

            simulated[i][j] = y_obs
    np.save(outDir + lcname[:-4], simulated)
    print('Done')

In [43]:
for ith in range(2,100):
    sim_Fig2_data(realLcs[ith])

 4.01 < tau_in < 60137.46  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.23 < tau_in < 78403.25  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 4.70 < tau_in < 70472.32  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.14 < tau_in < 77146.76  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.21 < tau_in < 78118.11  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.14 < tau_in < 77146.53  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 4.45 < tau_in < 66691.13  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.16 < tau_in < 77371.06  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.15 < tau_in < 77204.98  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50


10
20
30
40
50
60
70
80
90
Done
 5.06 < tau_in < 75858.58  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.51 < tau_in < 82666.27  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.05 < tau_in < 75813.25  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.21 < tau_in < 78207.94  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.23 < tau_in < 78493.24  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.23 < tau_in < 78493.08  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 4.70 < tau_in < 70533.73  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.15 < tau_in < 77281.03  [days]
Input  SF_inf: 0.2 sigma: 0.1414213562373095
0
10
20
30
40
50
60
70
80
90
Done
 5.51 < tau_in < 82666.63  [days]
Input  SF_inf: 0.2 sigma: 0.14

In [5]:
outDir

'../dp2/Fig2_simLC_SDSS-PS1/'